## Preambule

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import tqdm
import xarray as xr
import yaml

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
current_dir = Path.cwd()
with open(current_dir / 'input.yml') as file:
    settings = yaml.load(file, Loader=yaml.FullLoader)

## Read

In [3]:
path_save = settings['paths']['data']['datadrive']
xr_dataread = xr.open_dataset(path_save + "startyear_2021/xr_dataread.nc", engine='netcdf4')
all_countries_iso = np.load(path_save + "all_countries.npy", allow_pickle=True)
focusregs = np.array(xr_dataread.Region)
rules = ['GF', 'PC', 'PCC', 'PCB_lin', 'PCB', 'ECPC', 'AP', 'GDR']

## Reduction percentages for EZK

In [6]:
# for t in [2030, 2035, 2040]:
#     ds = xr.open_dataset(path_save + "xr_alloc_"+str(t)+".nc")
#     cur = xr_dataread.GHG_hist.sel(Time=1990)
#     ds = -(cur-ds)/cur
#     ds.sel(Discount_factor=0, Historical_startyear=1990, Capability_threshold='Th', RCI_weight='Half', Scenario='SSP2', Convergence_year=2050, Risk=0.5, NegEmis=0.5).drop_vars(['Scenario', 'Convergence_year', 'Discount_factor', 'Historical_startyear', 'Capability_threshold', 'RCI_weight', 'Time', 'source', 'Version', 'variable']).to_dataframe().to_csv(path_save + "Reductions_"+str(t)+".csv")

## Reduction percentages by 2040 w.r.t. 2015 (Paris):

In [7]:
# for t in [2030, 2035, 2040]:
#     ds = xr.open_dataset(path_save + "xr_alloc_"+str(t)+".nc")
#     cur = xr_dataread.GHG_hist.sel(Time=2015)
#     ds = -(cur-ds)/cur
#     ds.sel(Discount_factor=0, Historical_startyear=1990, Capability_threshold='Th', RCI_weight='Half', Scenario='SSP2', Convergence_year=2050, Risk=0.5, NegEmis=0.5, Temperature=1.6, NonCO2red=0.5, Timing='Immediate'
#            ).drop_vars(['Scenario', 'Convergence_year', 'Discount_factor', 'Historical_startyear', 'Capability_threshold', 'RCI_weight', 'Time', 'source', 'Version', 'variable', 'NegEmis', 'Temperature', 'Risk', 'NonCO2red', 'Timing']).to_dataframe().to_csv(path_save + "Reductions_"+str(t)+"_wrt2015.csv")

## Save allocations by year

In [4]:
path = 'K:/Data/Data_effortsharing/DataUpdate_ongoing/startyear_2021/'
for year in np.arange(2022, 2101, 1):#np.array([2030, 2040, 2050]):#, 2060, 2070, 2080, 2090, 2100]):
    print("Starting with", year)
    for cty_i, cty in tqdm(enumerate(focusregs)):
        ds = xr.open_dataset(path + "Allocations_GHG_incl/xr_alloc_"+cty+".nc").sel(Time=year).expand_dims(Region=[cty])
        if cty_i == 0:
            xrt = ds.copy()
        else:
            xrt = xr.merge([xrt, ds])
        ds.close()
    xrt.astype("float32").to_netcdf(path + "/Aggregated_files/xr_alloc_"+str(year)+"_GHG_incl.nc", format='NETCDF4')

Starting with 2022


207it [09:22,  2.72s/it]


Starting with 2023


207it [08:39,  2.51s/it]


Starting with 2024


...

207it [08:30,  2.47s/it]


Starting with 2099


207it [08:30,  2.47s/it]


Starting with 2100


207it [08:38,  2.51s/it]


## Allocations by year CO2

In [7]:
for year in np.arange(2025, 2101, 5):#np.array([2030, 2040, 2050]):#, 2060, 2070, 2080, 2090, 2100]):
    print("Starting with", year)
    for cty_i, cty in tqdm(enumerate(focusregs)):
        ds = xr.open_dataset(path_alloc + "xr_alloc_"+cty+"_CO2.nc").sel(Time=year).expand_dims(Region=[cty])
        if cty_i == 0:
            xrt = ds.copy()
        else:
            xrt = xr.merge([xrt, ds])
        ds.close()
    xrt.astype("float32").to_netcdf(path_save + "xr_alloc_"+str(year)+"_CO2.nc", format='NETCDF4')

Starting with 2025


207it [12:57,  3.75s/it]


Starting with 2030


207it [12:14,  3.55s/it]


Starting with 2035


...

207it [11:18,  3.28s/it]


Starting with 2095


207it [11:25,  3.31s/it]


Starting with 2100


207it [11:17,  3.27s/it]


Trajectory emissions

In [12]:
# xrs = []
# for cty in tqdm(all_countries_iso[:5]):
#     xrs.append(xr.open_dataset(path_save + "Allocations/xr_alloc_"+cty+".nc").sel(Temperature=1.5, Risk=0.5, NonCO2red=0.5, Timing='Immediate', NegEmis=0.5, Time=np.arange(params['start_year_analysis'], 2101))['GF'].expand_dims(Region=[cty]))

In [5]:
path = 'K:/Data/Data_effortsharing/DataUpdate_ongoing/startyear_2021/'
for temp_i, temp in enumerate([1.5, 1.6, 2.0]):
    for cty_i, cty in tqdm(enumerate(focusregs)):
        ds = xr.open_dataset(path + "Allocations_GHG_incl/xr_alloc_"+cty+".nc").sel(Temperature=temp, Risk=[0.5, 0.5, 0.33][temp_i], Time=np.arange(2021, 2101)).expand_dims(Region=[cty])
        if cty_i == 0:
            xrt = ds.copy()
        else:
            xrt = xr.merge([xrt, ds])
        ds.close()
    xrt.astype("float32").to_netcdf(path + "Aggregated_files/xr_traj_"+['t15_r50', 't16_r50', 't20_r67'][temp_i]+"_GHG_incl.nc", format='NETCDF4')

207it [20:10,  5.85s/it]
207it [19:51,  5.76s/it]
207it [19:41,  5.71s/it]
